# How to reprocess ODFs to generate calibrated and concatenated EPIC event lists

---

#### Introduction
This thread illustrates how to reprocess Observation Data Files (ODFs) to obtain calibrated and concatenated event lists.
#### Expected Outcome
The user will obtain calibrated and concatenated event lists which can be directly used to generate scientific products (images, spectra, light curves) through the SAS tasks [<tt>evselect</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/evselect/index.html) or [<tt>xmmselect</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/xmmselect/index.html).
#### SAS Tasks to be Used

- `emproc`[(Documentation for emproc)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emproc/index.html "emproc Documentation")
- `epproc`[(Documentation for epproc)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html "epproc Documentation")

#### Prerequisites
On SciServer both SAS and HEASOFT are automatically initialized when opening a container and starting a kernal.
#### Useful Links

- [`pysas` Documentation](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/pysas/index.html "pysas Documentation")
- [Common SAS Threads](https://www.cosmos.esa.int/web/xmm-newton/sas-threads/index.html "SAS Threads")
- [Users' Guide to the XMM-Newton Science Analysis System (SAS)](https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/sas_usg/USG/SASUSG.html "Users' Guide")
- [The XMM-Newton ABC Guide](https://heasarc.gsfc.nasa.gov/docs/xmm/abc/ "ABC Guide")

#### Caveats
The following assumes this notebook is run from the <tt>(xmmsas)</tt> environment on SciServer. You should see <tt>(xmmsas)</tt> at the top right of the notebook. If not, click there and select <tt>(xmmsas)</tt>. This tutorial assumes you are familiar with at least the Short pySAS Introduction Jupyter Notebook (<a href="./xmm-pysas-intro-short.ipynb">Short pySAS Introduction</a>).

##### Last Reviewed: _12 December 2023, for SAS v21_
##### Last Updated: _12 December 2023_
---

## Procedure
Run the EPIC reduction meta-tasks.

    For EPIC-MOS:
        emproc

    and for EPIC-pn:
        epproc

That's it! The default values of these meta-tasks are appropriate for most practical cases. You may have a look at the next section in this thread to learn how to perform specific reduction sub-tasks using [emproc](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emproc/index.html) or [epproc](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html).

The files produced by [epproc](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html) are the following:

 - `????_??????????_AttHk.ds`, the reconstructed attitude file
 - `????_??????????_EPN_????_01_Badpixels.ds`, one table per reduced CCD containing the bad pixels
 - `????_??????????_EPN_????_ImagingEvts.ds`, the calibrated and concatenated event list, which shall be used as an input to extract scientific products via [evselect](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/evselect/index.html) or [xmmselect](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/xmmselect/index.html).
    
The files produced by [emproc](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emproc/index.html) are conceptually the same. The main difference in the naming convention is that the string `EPN` is replaced by `EMOS1` and `EMOS2` for each EPIC-MOS camera, respectively.
___

<div class="alert alert-block alert-warning">
    <b>Warning:</b> Make sure you are running this notebook inside your <tt>persistent</tt> directory. If it is not inside the <tt>persistent</tt> directory your data will be lost upon exiting SciServer.
</div>

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook uses a development version of <tt>pysas</tt>, known as <tt>gofpysas</tt>. The main difference between <tt>gofpysas</tt> and <tt>pysas</tt> is the <tt>odf</tt> object, which contains additional functions to simplify the proccess of downloading and calibrating data, along with storing links to various directories and files. <tt>gofpysas</tt> will also automatically initialize and start SAS when imported into Python. <tt>gofpysas</tt> was developed specifically for SciServer, and all capabilites will be incorporated into a future release of <tt>pysas</tt>.
</div>

In [ ]:
import os
import gofpysas as pysas
data_dir = os.join.path(os.getcwd(),'xmm_data')
obsid = '0104860501'

In [ ]:
odf = pysas.odfcontrol.ODFobject(obsid)
odf.basic_setup(data_dir=data_dir,overwrite=False,repo='sciserver')

### Visualize the contents of the event files just created

The odf object contains a dictionary with the path and filename for important output files created by `basic_setup`.

In [ ]:
file_names = list(odf.files.keys())
print(file_names)
for name in file_names: print(odf.files[name])

The next blocks can be used to visualize the contents of the newly created event files. A simple selection criteria is impossed for illustration purposes. The thread <i>How to filter EPIC event lists for flaring particle background</i> will show how to use more complex filtering expressions by using the SAS task [evselect](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/evselect/index.html).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from matplotlib.colors import LogNorm

In [ ]:
# For display purposes only, define a minimum filtering criteria for EPIC-pn

pn_pattern   = 4        # pattern selection
pn_pi_min    = 300.     # Low energy range eV
pn_pi_max    = 12000.   # High energy range eV
pn_flag      = 0        # FLAG

# For display purposes only, define a minimum filtering criteria for EPIC-MOS

mos_pattern   = 12      # pattern selection
mos_pi_min    = 300.    # Low energy range eV
mos_pi_max    = 20000.  # High energy range eV
mos_flag      = 0       # FLAG

plt.figure(figsize=(15,25))

pl=1

pnevt_list = odf.files['pnevt_list']
m1evt_list = odf.files['m1evt_list']
m2evt_list = odf.files['m2evt_list']

evts=len(pnevt_list)+len(m1evt_list)+len(m2evt_list)
if len(pnevt_list) >0:
    for x in pnevt_list:
        hdu_list = fits.open(x, memmap=True)
        evt_data = Table(hdu_list[1].data)
        
        mask = ((evt_data['PATTERN'] <= pn_pattern) &
                (evt_data['FLAG'] == pn_flag) &
                (evt_data['PI'] >= pn_pi_min) &
                (evt_data['PI'] <= pn_pi_max))
        print("Events in event file" + " " + x + ": " + str(len(evt_data)) + "\n")
        print("Events in filtered event file" + " " + x + ": " + str(np.sum(mask)) + "\n")

# Create Events image        

        xmax=np.amax(evt_data['X']) 
        xmin=np.amin(evt_data['X']) 
        xmid=(xmax-xmin)/2.+xmin
        ymax=np.amax(evt_data['Y']) 
        ymin=np.amin(evt_data['Y'])
        xbin_size=80
        ybin_size=80
        NBINS = (int((xmax-xmin)/xbin_size),int((ymax-ymin)/ybin_size))
   
        plt.subplot(evts, 2, pl)

        img_zero_mpl = plt.hist2d(evt_data['X'], evt_data['Y'], NBINS, cmap='GnBu', norm=LogNorm())

        cbar = plt.colorbar(ticks=[10.,100.,1000.])
        cbar.ax.set_yticklabels(['10','100','1000'])
        
        plt.title(x)
        plt.xlabel('x')
        plt.ylabel('y')
   
        pl=pl+1
    
# Create Filtered Events image

        xmax=np.amax(evt_data['X'][mask]) 
        xmin=np.amin(evt_data['X'][mask]) 
        xmid=(xmax-xmin)/2.+xmin
        ymax=np.amax(evt_data['Y'][mask]) 
        ymin=np.amin(evt_data['Y'][mask])
        xbin_size=80
        ybin_size=80
        NBINS = (int((xmax-xmin)/xbin_size),int((ymax-ymin)/ybin_size))
   
        plt.subplot(evts, 2, pl)

        img_zero_mpl = plt.hist2d(evt_data['X'][mask], evt_data['Y'][mask], NBINS, cmap='GnBu', norm=LogNorm())

        cbar = plt.colorbar(ticks=[10.,100.,1000.])
        cbar.ax.set_yticklabels(['10','100','1000'])
        
        plt.title(x)
        plt.xlabel('x')
        plt.ylabel('y')

        txt=("PATTERN <= " + str(pn_pattern) + 
            " : " + str(pn_pi_min) + " <= E(eV) <= " + str(pn_pi_max) + 
            " : " + " FLAG = " + str(pn_flag))
        plt.text(xmid, ymin+0.1*(ymax-ymin), txt, ha='center')
    
        pl=pl+1

        hdu_list.close()
    
if len(m1evt_list) >0:
    for x in m1evt_list:
        hdu_list = fits.open(x, memmap=True)
        evt_data = Table(hdu_list[1].data)

        mask = ((evt_data['PATTERN'] <= mos_pattern) &
                (evt_data['FLAG'] == mos_flag) &
                (evt_data['PI'] >= mos_pi_min) &
                (evt_data['PI'] <= mos_pi_max))
        print("Events in event file" + " " + x + ": " + str(len(evt_data)) + "\n")
        print("Events in filtered event file" + " " + x + ": " + str(np.sum(mask)) + "\n")

# Create Events image            

        xmax=np.amax(evt_data['X']) 
        xmin=np.amin(evt_data['X']) 
        xmid=(xmax-xmin)/2.+xmin
        ymax=np.amax(evt_data['Y']) 
        ymin=np.amin(evt_data['Y'])
        xbin_size=80
        ybin_size=80
        NBINS = (int((xmax-xmin)/xbin_size),int((ymax-ymin)/ybin_size))
   
        plt.subplot(evts, 2, pl)

        img_zero_mpl = plt.hist2d(evt_data['X'], evt_data['Y'], NBINS, cmap='GnBu', norm=LogNorm())

        cbar = plt.colorbar(ticks=[10.,100.,1000.])
        cbar.ax.set_yticklabels(['10','100','1000'])
        
        plt.title(x)
        plt.xlabel('x')
        plt.ylabel('y')
 
        pl=pl+1
    
# Create Filtered Events image

        xmax=np.amax(evt_data['X'][mask]) 
        xmin=np.amin(evt_data['X'][mask]) 
        xmid=(xmax-xmin)/2.+xmin
        ymax=np.amax(evt_data['Y'][mask]) 
        ymin=np.amin(evt_data['Y'][mask])
        xbin_size=80
        ybin_size=80
        NBINS = (int((xmax-xmin)/xbin_size),int((ymax-ymin)/ybin_size))
   
        plt.subplot(evts, 2, pl)

        img_zero_mpl = plt.hist2d(evt_data['X'][mask], evt_data['Y'][mask], NBINS, cmap='GnBu', norm=LogNorm())

        cbar = plt.colorbar(ticks=[10.,100.,1000.])
        cbar.ax.set_yticklabels(['10','100','1000'])
        
        plt.title(x)
        plt.xlabel('x')
        plt.ylabel('y')

        txt=("PATTERN <= " + str(mos_pattern) + 
            " : " + str(mos_pi_min) + " <= E(eV) <= " + str(mos_pi_max) +
            " : " + " FLAG = " + str(mos_flag))
        plt.text(xmid, ymin+0.1*(ymax-ymin), txt, ha='center')
 
        pl=pl+1
    
        hdu_list.close()
    
if len(m2evt_list) >0:
    for x in m2evt_list:
        hdu_list = fits.open(x, memmap=True)
        evt_data = Table(hdu_list[1].data)

        mask = ((evt_data['PATTERN'] <= mos_pattern) &
                (evt_data['FLAG'] == mos_flag) &
                (evt_data['PI'] >= mos_pi_min) &
                (evt_data['PI'] <= mos_pi_max))
        print("Events in event file" + " " + x + ": " + str(len(evt_data)) + "\n")
        print("Events in filtered event file" + " " + x + ": " + str(np.sum(mask)) + "\n")
        
# Create Events image

        xmax=np.amax(evt_data['X']) 
        xmin=np.amin(evt_data['X']) 
        xmid=(xmax-xmin)/2.+xmin
        ymax=np.amax(evt_data['Y']) 
        ymin=np.amin(evt_data['Y'])
        xbin_size=80
        ybin_size=80
        NBINS = (int((xmax-xmin)/xbin_size),int((ymax-ymin)/ybin_size))
   
        plt.subplot(evts, 2, pl)

        img_zero_mpl = plt.hist2d(evt_data['X'], evt_data['Y'], NBINS, cmap='GnBu', norm=LogNorm())

        cbar = plt.colorbar(ticks=[10.,100.,1000.])
        cbar.ax.set_yticklabels(['10','100','1000'])

        plt.title(x)
        plt.xlabel('x')
        plt.ylabel('y')
       
        pl=pl+1
    
# Create Filtered Events image    
       
        xmax=np.amax(evt_data['X'][mask]) 
        xmin=np.amin(evt_data['X'][mask]) 
        xmid=(xmax-xmin)/2.+xmin
        ymax=np.amax(evt_data['Y'][mask]) 
        ymin=np.amin(evt_data['Y'][mask])
        xbin_size=80
        ybin_size=80
        NBINS = (int((xmax-xmin)/xbin_size),int((ymax-ymin)/ybin_size))
   
        plt.subplot(evts, 2, pl)

        img_zero_mpl = plt.hist2d(evt_data['X'][mask], evt_data['Y'][mask], NBINS, cmap='GnBu', norm=LogNorm())

        cbar = plt.colorbar(ticks=[10.,100.,1000.])
        cbar.ax.set_yticklabels(['10','100','1000'])

        plt.title(x)
        plt.xlabel('x')
        plt.ylabel('y')
        
        txt=("PATTERN <= " + str(mos_pattern) + 
            " : " + str(mos_pi_min) + " <= E(eV) <= " + str(mos_pi_max) + 
            " : " + " FLAG = " + str(mos_flag))
        plt.text(xmid, ymin+0.1*(ymax-ymin), txt, ha='center')

        pl=pl+1

        hdu_list.close()
        


### How to accomplish specific reduction tasks

[<tt>emproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emproc/index.html) and [<tt>epproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html) are highly flexible tasks, which allow the user to perform a wide range of customized reduction tasks. Some [<tt>emproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emproc/index.html) examples are listed below. The same customized reduction tasks can be performed for the EPIC-pn as well, just by substituting [<tt>emproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emproc/index.html) with [<tt>epproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html) in the commands.

- If you want to reduce only one of the cameras (EPIC-MOS1 in the example):

    <tt>emproc selectinstruments=yes emos1=yes</tt>

In [ ]:
# SAS Command
cmd    = "emproc" # SAS task to be executed                  

# Arguments of SAS Command
inargs = ['selectinstruments=yes','emos1=yes']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
w(cmd, inargs).run()

- If you want to reduce only a subsample of exposures:

    <tt>emproc withinstexpids=yes instexpids="M1S001 M2S002"</tt>

In [ ]:
# SAS Command
cmd    = "emproc" # SAS task to be executed                  

# Arguments of SAS Command
inargs = ['withinstexpids=yes','instexpids="M1S001 M2S002"']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
w(cmd, inargs).run()

- If you want to reduce data from 1 CCD only (#4 and #5 in the example):

    <tt>emproc selectccds=yes ccd4=yes ccd5=yes</tt>

In [ ]:
# SAS Command
cmd    = "emproc" # SAS task to be executed                  

# Arguments of SAS Command
inargs = ['selectccds=yes','ccd4=yes','ccd5=yes']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
w(cmd, inargs).run()

- If you want to change the reference pointing for the calculation of the sky coordinates to a value of your choice:

    <tt>emproc referencepointing=user ra=34.65646 dec=-12.876546</tt>

In [ ]:
# SAS Command
cmd    = "emproc" # SAS task to be executed                  

# Arguments of SAS Command
inargs = ['referencepointing=user','ra=34.65646','dec=-12.876546']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
w(cmd, inargs).run()

- Please be aware that if you want to supply coordinates for the analysis of the EPIC-MOS Timing mode, the command is slightly different, e.g.:

    <tt>emproc withsrccoords=yes srcra=34.65646 srcdec=-12.876546</tt>

- If you want to filter the event list events, using an external Good Time Interval (GTI) file (see the corresponding thread on how to filter event files for flaring particle background by creating a GTI file):

    <tt>emproc withgtiset=yes gtiset=mygti.gti filterevents=yes</tt>

In [ ]:
# SAS Command
cmd    = "emproc" # SAS task to be executed                  

# Arguments of SAS Command
inargs = ['withgtiset=yes','gtiset=mygti.gti','filterevents=yes']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
w(cmd, inargs).run()

Parameters can be combined to accomplish simultaneously two or more of the above tasks during the same run.

The user is referred to the on-line documentation of [<tt>emproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emproc/index.html) and [<tt>epproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html) for a complete list of the available options.

### Reduction of EPIC-pn Timing Mode exposures

Most exposures in EPIC-pn Timing Mode are affected by X-ray Loading (XRL; cf. Sect.3.1 in Guainazzi et al., 2013, [XMM-SOC-CAL-TN-0083](http://xmm2.esac.esa.int/docs/documents/CAL-TN-0083.pdf)). Furthermore, a residual dependence of the energy scale on the total count rate is corrected through the "Rate-Dependent PHA" correction (Guainazzi, 2014, [XMM-CCF-REL-312](http://xmm2.esac.esa.int/docs/documents/CAL-SRN-0312-1-4.pdf)). In order to correct for these effects a set of default calibration settings have been identified. As of SAS v14.0, this is controlled by a single parameter within the tasks [<tt>epproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html) and [<tt>epchain</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epchain/index.html). This parameter is called <tt>withdefaultcal</tt> and is set to <tt>yes</tt> by default. Setting <tt>withdefaultcal=yes</tt> implies <tt>runepreject=yes withxrlcorrection=yes runepfast=no withrdpha=yes</tt>. So one shall run the EPIC-pn reduction meta-tasks as follows:

&emsp;&emsp;<tt>epproc </tt>

or:

&emsp;&emsp;<tt>epchain datamode=TIMING</tt>

For more information please refer to the documentation of [<tt>epproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html) and [<tt>epchain</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epchain/index.html).

### Reduction of EPIC-pn Burst Mode exposures

Most exposures in EPIC-pn Burst Mode are affected by X-ray Loading (XRL; cf. Sect.3.1 in Guainazzi et al., 2013, [XMM-SOC-CAL-TN-0083](http://xmm2.esac.esa.int/docs/documents/CAL-TN-0083.pdf)). Furthermore, a residual dependence of the energy scale on the total count rate is corrected through the "Rate-Dependent CTI" correction. In order to correct for these effects a set of default calibration settings have been identified. As of SAS v14.0, this is controlled by a single parameter within the tasks [<tt>epproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html) and [<tt>epchain</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epchain/index.html). This parameter is called <tt>withdefaultcal</tt> and is set to <tt>yes</tt> by default. Setting <tt>withdefaultcal=yes</tt> implies <tt>runepreject=yes withxrlcorrection=yes runepfast=yes withrdpha=no</tt>. So one shall run the EPIC-pn reduction meta-tasks as follows:

&emsp;&emsp;<tt>epproc burst=yes</tt>

Notice the inclusion of the extra parameter <tt>burst=yes</tt> in the call to [<tt>epproc</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epproc/index.html) and [<tt>epchain</tt>](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epchain/index.html) also needs an extra parameter:

&emsp;&emsp;<tt>epchain datamode=BURST</tt>

In [ ]:
# SAS Command
cmd    = "epchain" # SAS task to be executed                  

# Arguments of SAS Command
inargs = ['datamode=BURST']

print("   SAS command to be executed: "+cmd+", with arguments; \n")
inargs

In [ ]:
w(cmd,inargs).run()